# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,13.45,65,100,2.17,RU,1691614608
1,1,port-aux-francais,-49.3500,70.2167,2.65,78,92,12.29,TF,1691614781
2,2,mitu,1.1983,-70.1733,26.90,75,99,1.59,CO,1691614844
3,3,adamstown,-25.0660,-130.1015,18.95,69,43,9.62,PN,1691614844
4,4,bilibino,68.0546,166.4372,10.98,47,99,1.54,RU,1691614601


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
City_Humidity = city_data_df.hvplot.points("Lng","Lat",geo = True,tiles = "EsriImagery",frame_width = 800,frame_height = 600,size = "Humidity",scale = 0.5,color = "City")

# Display the map
City_Humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
good_temp_df = city_data_df.loc[city_data_df['Max Temp']>21]
better_temp_df = good_temp_df.loc[good_temp_df['Max Temp']<27]
low_wind_df = better_temp_df.loc[better_temp_df['Wind Speed']<4.5]
perfect_weather_df = low_wind_df.loc[low_wind_df['Cloudiness']==0]

# Drop any rows with null values
perfect_city_df = perfect_weather_df.dropna(how ='any')

# Display sample data
perfect_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,80,mammoth lakes,37.6485,-118.9721,25.45,10,0,3.13,US,1691614863
91,91,arish,31.1316,33.7984,25.74,69,0,1.59,EG,1691614865
117,117,detroit lakes,46.8172,-95.8453,23.38,59,0,2.57,US,1691614729
312,312,shanhaiguan,39.9988,119.7564,22.84,73,0,2.54,CN,1691614999
323,323,kashgar,39.4547,75.9797,26.56,29,0,4.00,CN,1691615001
358,358,massagno,46.0156,8.9453,21.48,78,0,0.89,CH,1691615071
362,362,broome,42.2506,-75.8330,26.44,67,0,3.09,US,1691615072
420,420,manghit,42.1156,60.0597,24.99,27,0,2.67,UZ,1691615079
433,433,varzelandia,-15.7014,-44.0275,24.00,34,0,3.03,BR,1691615081
454,454,mahabad,36.7631,45.7222,24.63,29,0,1.69,IR,1691615083


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
perfect_city_df.copy(deep=True)
hotel_df = perfect_city_df.drop(columns=['City_ID','Max Temp','Cloudiness','Wind Speed','Date'])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
80,mammoth lakes,37.6485,-118.9721,10,US,
91,arish,31.1316,33.7984,69,EG,
117,detroit lakes,46.8172,-95.8453,59,US,
312,shanhaiguan,39.9988,119.7564,73,CN,
323,kashgar,39.4547,75.9797,29,CN,
358,massagno,46.0156,8.9453,78,CH,
362,broome,42.2506,-75.8330,67,US,
420,manghit,42.1156,60.0597,27,UZ,
433,varzelandia,-15.7014,-44.0275,34,BR,
454,mahabad,36.7631,45.7222,29,IR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {"filter":"","bias":""}

# accommodation.hotel	 

# Print a message to follow up the hotel search
print("Starting hotel search")
categories = "categories=accommodation.hotel"
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'filter=circle:{lon},{lat},{radius}'
    params["bias"] = f'bias=proximity:{lon},{lat}'
    # Set base URL
    base_url = f'https://api.geoapify.com/v2/places?apiKey={geoapify_key}&{categories}&{params["bias"]}&{params["filter"]}'

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
arish - nearest hotel: سينا
detroit lakes - nearest hotel: Graystone Hotel
shanhaiguan - nearest hotel: Bo Yi Business Hotel
kashgar - nearest hotel: 深航国际酒店
massagno - nearest hotel: Villa Sassa
broome - nearest hotel: No hotel found
manghit - nearest hotel: Mang'it miymanxanasi
varzelandia - nearest hotel: No hotel found
mahabad - nearest hotel: هتل آپارتمان مکریان
uruzgan - nearest hotel: No hotel found
gharyan - nearest hotel: فندق الرابطة
chenaran - nearest hotel: No hotel found
oltinko'l - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
80,mammoth lakes,37.6485,-118.9721,10,US,Travelodge by Wyndham Mammoth Lakes
91,arish,31.1316,33.7984,69,EG,سينا
117,detroit lakes,46.8172,-95.8453,59,US,Graystone Hotel
312,shanhaiguan,39.9988,119.7564,73,CN,Bo Yi Business Hotel
323,kashgar,39.4547,75.9797,29,CN,深航国际酒店
358,massagno,46.0156,8.9453,78,CH,Villa Sassa
362,broome,42.2506,-75.8330,67,US,No hotel found
420,manghit,42.1156,60.0597,27,UZ,Mang'it miymanxanasi
433,varzelandia,-15.7014,-44.0275,34,BR,No hotel found
454,mahabad,36.7631,45.7222,29,IR,هتل آپارتمان مکریان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
Hotel_Name = hotel_df.hvplot.points("Lng","Lat",geo = True,tiles = "EsriImagery",frame_width = 800,frame_height = 600,scale = 0.5,color = "City",hover_cols=['Hotel Name'])

# Display the map
Hotel_Name

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)